# Connecting DB

In [1]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd

def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        # Set the role
        cursor.execute("SET ROLE agl_dwh_read;")
        # Execute the main query
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

def fetch_query(connection, query):
    cursor = connection.cursor()
    try:
        # Set the role
        cursor.execute("SET ROLE agl_dwh_read;")
        # Execute the main query and fetch results
        cursor.execute(query)
        result = cursor.fetchall()
        # Convert the result into a pandas DataFrame
        df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
        return df
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    finally:
        cursor.close()

# Replace these variables with your database details
db_name = "agl_dwh"
db_user = "agoyal"
db_password = "Aashi2109@"
db_host = "agl-dwh-db.czwfcxopgeqv.us-east-1.rds.amazonaws.com"
db_port = "5432"

connection = create_connection(db_name, db_user, db_password, db_host, db_port)


Connection to PostgreSQL DB successful


Steps:
1) connect to dwh 
2) extract teh CW data from sql 
sql: 
SELECT 
    '', --operation_id populated as hash in python
    'CW', --as [System],
    'A' as RecStatus,
    '',   --as [LOB],
    (SELECT TOP 1 staff.GS_FullName 
     FROM CustomsReporting.dbo.Cargowise_OrgStaffAssignments_PROD_REPORTING sa
     JOIN CustomsReporting.dbo.Cargowise_GLBStaff_PROD_REPORTING staff
     ON staff.GS_Code = (
            SELECT TOP 1 O8_GS_NKPersonResponsible 
            FROM CustomsReporting.dbo.Cargowise_OrgStaffAssignments_PROD_REPORTING sa_inner
            WHERE sa_inner.O8_OH = sa.O8_OH 
            AND sa_inner.O8_Role = 
                CASE 
                    WHEN EXISTS (
                        SELECT 1 
                        FROM CustomsReporting.dbo.Cargowise_OrgStaffAssignments_PROD_REPORTING sa2 
                        WHERE sa2.O8_OH = sa.O8_OH 
                        AND sa2.O8_Role = 'CUS'
                    ) THEN 'CUS'
                    ELSE 'ACT'
                END
            ORDER BY sa_inner.O8_SystemCreateTimeUTC DESC  -- Ensure only one value is selected
        )
     WHERE sa.O8_OH = OH.OH_PK
     ORDER BY sa.O8_SystemCreateTimeUTC DESC) AS operator, --Operator
    '', --customercsr  
    CASE WHEN JS_RS_NKServiceLevel = 'AWS' THEN 'All Water'
             WHEN JS_RS_NKServiceLevel = 'MLB' THEN 'MLB' 
             ELSE NULL END AS ShipMode, --case statement to return either 'MLB, All Water'
    '', -- as  [DeliveryType]
    jk.JK_UniqueConsignRef as ref_id, --as [REFID], --C OR S USED IN JOINS
    jk.JK_BookingReference, -- as [BookingNo],
    jk.JK_MasterBillNum, --MBL
    JS.JS_HouseBill,
    '', --SONumber
    CAST(OH.OH_PK AS VARCHAR(100)) AS to_company_id,
    OH.OH_FullName as to_company, --as ToC,  
    OH.OH_FullName as consignee, -- as Consignee,
    '', --notify
    (SELECT OH_FullName FROM CustomsReporting.[dbo].[Cargowise_ORGHEADER_PROD_REPORTING] OH WHERE OH.OH_PK = JS.JS_OH_ImportBroker) AS Broker, --broker
    '', --NVO
    '', --shipline
    (SELECT oh.OH_CODE
     FROM CustomsReporting.[dbo].[Cargowise_OrgAddress_PROD_REPORTING] oa 
     JOIN CustomsReporting.[dbo].[Cargowise_ORGHEADER_PROD_REPORTING] oh ON oh.OH_PK = oa.OA_OH
     WHERE oa.OA_PK = jk.JK_OA_ShippingLineAddress) AS [SCAC],
    jwConsolMother.JW_Vessel, -- as [Vessel],
    jwConsolMother.JW_VoyageFlight, -- as [Voyage],
    jwConsolETD.JW_Vessel, -- as [Feeder],
    jwConsolETD.JW_VoyageFlight, -- as [FVOY],
    (SELECT RL_PortName + ', ' + RL_RN_NKCountryCode FROM CustomsReporting.[dbo].[Cargowise_RefUNLOCO_PROD_REPORTING] rl WHERE rl.RL_Code = jk.JK_RL_NKLoadPort) AS LoadingPort,
    (SELECT RL_PortName + ', ' + RL_RN_NKCountryCode FROM CustomsReporting.[dbo].[Cargowise_RefUNLOCO_PROD_REPORTING] rl WHERE rl.RL_Code = JS.JS_RL_NKOrigin), --CW origin
    (SELECT RL_PortName + ', ' + RL_RN_NKCountryCode FROM CustomsReporting.[dbo].[Cargowise_RefUNLOCO_PROD_REPORTING] rl WHERE rl.RL_Code = JK_RL_NKDischargePort) AS Destination, --CW dest
    (SELECT RL_PortName + ', ' + RL_RN_NKCountryCode FROM CustomsReporting.[dbo].[Cargowise_RefUNLOCO_PROD_REPORTING] rl WHERE rl.RL_Code = jwConsolMother.JW_RL_NKDiscPort), -- as [InPort],
    (SELECT TOP 1 CAST(CE_EntryNum AS VARCHAR(1000)) FROM CustomsReporting.[dbo].[Cargowise_CUSENTRYNUM_PROD_REPORTING] WHERE CE_EntryType = 'IT' AND CE_ParentID = JS_PK), -- as [InITNo],
    (SELECT RL_PortName + ', ' + RL_RN_NKCountryCode FROM CustomsReporting.[dbo].[Cargowise_RefUNLOCO_PROD_REPORTING] rl WHERE rl.RL_Code = JS.JS_RL_NKDestination) AS FinalDest,
    '', --incoterms
    '', -- as [CstmsRlsdBy],
    '',-- as [EntryNo],
    '0',-- as [CntrRls],
    '', -- as [DetailComment],
    '', -- as [AN_Remark],
    '', -- as [TradecardFCR],
    JK_RL_NKLOadPort AS LoadingPortID,
    JS.JS_RL_NKOrigin AS OriginPort,
    JK_RL_NKDischargePort AS DestinationPort,
    --Dates--
    NULL AS [CarrierBookingSubmitted],
    NULL AS [CargoReadyDate],
    NULL AS [CarrierBookingConfirmed],
    NULL AS [ERD],
    NULL AS [MBLDate], --eDocs tab    
    NULL AS [DocCutoffDate],
    NULL AS [CutOfDate],
    NULL AS [PORETA],
    NULL AS [PORATA],
    jwConsolETD.JW_ETD AS loading_etd,
    jwConsolETD.JW_ATD AS [ActDeptDate],
    NULL AS [DocCompleted],
    jwConsolMother.JW_ETA AS [DischargeETA],
    jwConsolMother.JW_ATA AS [ActDischargePort],    
    (SELECT MIN(CH_EntrySubmittedDate) 
     FROM CustomsReporting.[dbo].[Cargowise_CUSENTRYHEADER_PROD_REPORTING]
     WHERE CH_JE IN (
                    SELECT CH_JE 
                    FROM CustomsReporting.[dbo].[Cargowise_CUSENTRYHEADER_PROD_REPORTING] ch
                    JOIN CustomsReporting.[dbo].[Cargowise_JobDeclaration_PROD_REPORTING] je 
                    ON je.JE_PK = ch.CH_JE
                    WHERE je.JE_JS = js.JS_PK
                    ) 
    ) AS [CstmsRlsdDate],
    JK_SystemCreateTimeUtc AS CreateDate
FROM CustomsReporting.[dbo].[Cargowise_JobConsol_PROD_REPORTING] jk
JOIN CustomsReporting.[dbo].[Cargowise_JobConShipLink_PROD_REPORTING] l ON l.JN_JK = jk.JK_PK
JOIN CustomsReporting.[dbo].[Cargowise_JobShipment_PROD_REPORTING] js ON js.JS_PK = l.JN_JS
JOIN CustomsReporting.[dbo].[Cargowise_JobDocAddress_PROD_REPORTING] E2 ON E2.E2_ParentID = JS.JS_PK AND E2_AddressType = 'CED'
JOIN CustomsReporting.[dbo].[Cargowise_OrgAddress_PROD_REPORTING] OA ON OA.OA_PK = E2.E2_OA_Address
JOIN CustomsReporting.[dbo].[Cargowise_ORGHEADER_PROD_REPORTING] OH ON OH.OH_PK = OA.OA_OH
LEFT OUTER JOIN CustomsReporting.[dbo].[cwJWConsolETDViewCustomsReporting] jwConsolETD ON jwConsolETD.JW_ParentGuid = jk.JK_PK
LEFT OUTER JOIN CustomsReporting.[dbo].[cwJWConsolMotherViewCustomsReporting] jwConsolMother ON jwConsolMother.JW_ParentGuid = jk.JK_PK    
WHERE js.JS_TransportMode = 'SEA' 
AND jk.JK_SystemCreateTimeUtc >= '2021-01-01 00:00:00.000'
AND YEAR(jwConsolETD.JW_ETD) = 2024 -- Filter for records where the LoadingETD is in the year 2024
ORDER BY jwConsolETD.JW_ETD ASC;

3) extarct this using ODBC connection in excel
4) read that here in python: store in df_cw
5) fetching data of IE from dwh and stor ein :df_ie
6) columns_to_keep = ['ref_id', 'operator','loading_etd','to_company_id','to_company','consignee'] and stpr ein merged dataframe
7) join container table and get results pu tin the excel file that will be sent out
8) to get acc details, extract that using merged df into another excel and copy paste in the main excel to be used 

# Reading CW op data

In [2]:
df_cw=pd.read_excel('/Users/aashigoyal/Desktop/vscode/aashimain/agl/main_cw_op.xlsx')

In [3]:
df_cw

,(No column name),(No column name).1,RecStatus,(No column name).2,operator,(No column name).3,ShipMode,(No column name).4,ref_id,JK_BookingReference,...,CutOfDate,PORETA,PORATA,loading_etd,ActDeptDate,DocCompleted,DischargeETA,ActDischargePort,CstmsRlsdDate,CreateDate
0,NaN,CW,A,NaN,Benjamin Walker,NaN,All Water,NaN,C2401S089115,NaN,...,NaN,NaN,NaN,2024-01-01 00:00:00,2024-01-01 00:00:00,NaN,2024-01-25 08:00:00,2024-01-25 04:17:00,2024-01-20 00:47:00,2024-01-08 18:39:00
1,NaN,CW,A,NaN,Kelly Anderson,NaN,All Water,NaN,C2401S088300,NaN,...,NaN,NaN,NaN,2024-01-01 00:00:00,2024-01-01 00:19:00,NaN,2024-01-17 15:30:00,2024-01-17 15:18:00,2024-01-12 17:11:00,2024-01-02 17:45:00
2,NaN,CW,A,NaN,Cara Poor,NaN,All Water,NaN,C2401S088637,NaN,...,NaN,NaN,NaN,2024-01-01 00:00:00,2024-01-02 07:42:00,NaN,2024-02-04 00:00:00,2024-02-03 17:43:00,NaT,2024-01-02 23:48:00
3,NaN,CW,A,NaN,Ashley Schafer,NaN,All Water,NaN,C2401S088628,NaN,...,NaN,NaN,NaN,2024-01-01 00:00:00,2024-01-01 03:05:00,NaN,2024-02-10 05:30:00,2024-02-10 08:06:00,2024-02-05 08:17:00,2024-01-02 23:47:00
4,NaN,CW,A,NaN,Michelle Ford,NaN,All Water,NaN,C2312S087714,NaN,...,NaN,NaN,NaN,2024-01-01 00:00:00,2023-12-25 11:37:00,NaN,2024-03-13 08:00:00,2024-01-24 15:54:00,2024-01-23 10:16:00,2023-12-20 20:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21879,NaN,CW,A,NaN,Ashley Schafer,NaN,All Water,NaN,C2409S110152,NaN,...,NaN,NaN,NaN,2024-09-23 13:00:00,NaT,NaN,2024-11-12 16:00:00,NaT,NaT,2024-09-16 13:54:00
21880,NaN,CW,A,NaN,Cara Poor,NaN,All Water,NaN,C2409S109788,NaN,...,NaN,NaN,NaN,2024-09-24 13:00:00,NaT,NaN,2024-10-26 15:00:00,NaT,NaT,2024-09-09 16:08:00
21881,NaN,CW,A,NaN,Michelle Ford,NaN,All Water,NaN,C2409S110205,NaN,...,NaN,NaN,NaN,2024-09-26 23:00:00,NaT,NaN,2024-11-03 08:00:00,NaT,NaT,2024-09-16 13:57:00
21882,NaN,CW,A,NaN,Michelle Ford,NaN,All Water,NaN,C2409S110218,NaN,...,NaN,NaN,NaN,2024-09-26 23:00:00,NaT,NaN,2024-10-19 04:00:00,NaT,NaT,2024-09-16 13:58:00


In [4]:
# fetching data of IE from dwh 
query_fetch = """ select * from dbo.operation where "system" ='IE' and rec_status !='D' and EXTRACT(YEAR FROM loading_etd)=2024 """
results = fetch_query(connection, query_fetch)
print(results)

            ref_id                        hbl  \
0      50240400074                              
1      20240200041  460814044NGB(SNGB6560672)   
2      50240100078                              
3      50240100080                              
4      50240100105                              
...            ...                        ...   
11331  50240100055                              
11332  50240100056                              
11333  50240100057                              
11334  50240100058                              
11335  50240100116                              

                               operation_id system rec_status           lob  \
0      344dfa45-863f-5ec1-b82b-c4c35a914ae8     IE          A  Ocean Export   
1      b073bc76-8175-5bae-af55-cc893d3dd50a     IE          A  Ocean Import   
2      fdf63ae1-0916-5f50-8870-45088c09f485     IE          A  Ocean Export   
3      429e2878-559a-5d62-8728-b4819fa24253     IE          A  Ocean Export   
4      d0944430-

In [5]:
df_ie=pd.DataFrame(results)

In [6]:
df_ie

,ref_id,hbl,operation_id,system,rec_status,lob,operator,customer_csr,ship_mode,delivery_type,...,por_ata,loading_etd,act_dept_date,doc_completed,discharge_eta,act_discharge_port,cstms_rlsd_date,create_date,origin_type,destination_type
0,50240400074,,344dfa45-863f-5ec1-b82b-c4c35a914ae8,IE,A,Ocean Export,Gage Foster,,All Water,,...,NaT,2024-04-09,NaT,NaT,NaT,NaT,NaT,2024-04-09,Foreign,Foreign
1,20240200041,460814044NGB(SNGB6560672),b073bc76-8175-5bae-af55-cc893d3dd50a,IE,A,Ocean Import,Gage Foster,,All Water,,...,NaT,2024-01-24,NaT,NaT,2024-01-31,NaT,NaT,2024-02-01,Foreign,Foreign
2,50240100078,,fdf63ae1-0916-5f50-8870-45088c09f485,IE,A,Ocean Export,Christina Hutchens,,All Water,,...,NaT,2024-01-09,NaT,NaT,NaT,NaT,NaT,2024-01-09,Foreign,Foreign
3,50240100080,,429e2878-559a-5d62-8728-b4819fa24253,IE,A,Ocean Export,Christina Hutchens,,All Water,,...,NaT,2024-01-09,NaT,NaT,NaT,NaT,NaT,2024-01-09,Foreign,Foreign
4,50240100105,,d0944430-ccb3-5d8c-bcfb-fc807dfb0b02,IE,A,Ocean Export,Christina Hutchens,,All Water,,...,NaT,2024-01-12,NaT,NaT,NaT,NaT,NaT,2024-01-12,Foreign,Foreign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11331,50240100055,,973a017f-1c28-513b-aa15-ca88e14b50cc,IE,A,Ocean Export,Christina Hutchens,,All Water,,...,NaT,2024-01-04,NaT,NaT,NaT,NaT,NaT,2024-01-04,Foreign,Foreign
11332,50240100056,,f0049e23-d26d-5c07-bf4a-e214b2689a9f,IE,A,Ocean Export,Christina Hutchens,,All Water,,...,NaT,2024-01-04,NaT,NaT,NaT,NaT,NaT,2024-01-04,Foreign,Foreign
11333,50240100057,,fa7e8b4c-6e1d-5bcb-8cb9-ad75874ea00f,IE,U,Ocean Export,Terry Barbour,,All Water,,...,2024-02-14,2024-02-23,2024-02-23,NaT,2024-04-11,2024-04-11,NaT,2024-01-05,Foreign,Foreign
11334,50240100058,,36332aa8-c7aa-5646-bd2e-ea6af90b9344,IE,A,Ocean Export,Kim Jones,,All Water,,...,2024-07-22,2024-07-25,2024-07-26,NaT,2024-09-04,2024-09-02,NaT,2024-01-05,Foreign,Foreign


In [7]:
columns_to_keep = ['ref_id', 'operator','loading_etd','to_company_id','to_company','consignee']

df_cw_selected = df_cw[columns_to_keep]
df_ie_selected = df_ie[columns_to_keep]

df_merged_op = pd.concat([df_cw_selected, df_ie_selected], ignore_index=True)
print(df_merged_op)


             ref_id            operator loading_etd  \
0      C2401S089115     Benjamin Walker  2024-01-01   
1      C2401S088300      Kelly Anderson  2024-01-01   
2      C2401S088637           Cara Poor  2024-01-01   
3      C2401S088628      Ashley Schafer  2024-01-01   
4      C2312S087714       Michelle Ford  2024-01-01   
...             ...                 ...         ...   
33215   50240100055  Christina Hutchens  2024-01-04   
33216   50240100056  Christina Hutchens  2024-01-04   
33217   50240100057       Terry Barbour  2024-02-23   
33218   50240100058           Kim Jones  2024-07-25   
33219   50240100116       Terry Barbour  2024-02-09   

                              to_company_id                        to_company  \
0      3A9BC7D7-1C64-4193-8AF7-699CAF87D9FD  FUYAO NORTH AMERICA INCORPORATED   
1      3D155580-F014-43F3-BA91-0E5DAC86BDD0          EASTPOINT SPORTS LTD LLC   
2      F9EB87E7-7F10-4CAC-829E-A808D735A669                HERITAGE GROUP LLC   
3      5B8ABE21

In [8]:
# Replace operator name where to_company 
df_merged_op.loc[df_merged_op['to_company'] == 'AMERICAN SIGNATURE INC (ASI)', 'operator'] = 'ASI Team'
df_merged_op.loc[df_merged_op['to_company'] == 'SVP Sewing Brands, LLC', 'operator'] = 'David McKeen'
df_merged_op.loc[df_merged_op['to_company'] == 'ITG VOMA', 'operator'] = 'Michelle Ford'
df_merged_op.loc[df_merged_op['to_company'] == 'Igloo Products Corp.', 'operator'] = 'Gage Foster'

In [9]:
df_merged_op['operator'].fillna('missing', inplace=True)

/var/folders/cr/d45qtstd0ns7j8t3d1skhf_c0000gn/T/ipykernel_97273/692637306.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged_op['operator'].fillna('missing', inplace=True)


In [10]:
# Saving the DataFrame to an Excel file
excel_filename = 'output_merged_df.xlsx'
df_merged_op.to_excel(excel_filename, index=False, engine='openpyxl')

print(f'DataFrame merged is saved to {excel_filename}')

DataFrame merged is saved to output_merged_df.xlsx


In [11]:
# Load the container table into a DataFrame
query = "SELECT ref_id, container_id FROM dbo.container"
container_results=fetch_query(connection, query)
container_df=pd.DataFrame(container_results)

# Perform the join operation between the DataFrame in your notebook and the container DataFrame
merged_df = pd.merge(df_merged_op, container_df, on='ref_id', how='left')

# Convert the 'loading_etd' column to datetime format if it's not already
merged_df['loading_etd'] = pd.to_datetime(merged_df['loading_etd'], errors='coerce')

# Perform the aggregation and grouping on the merged DataFrame
result_df = merged_df.groupby([
    'operator',
    merged_df['loading_etd'].dt.year.rename('loading_year'),
    merged_df['loading_etd'].dt.month.rename('loading_month')
]).agg(
    container_count=('container_id', 'nunique'),
    ref_id_count=('ref_id', 'nunique')
).reset_index()

# Display the final DataFrame
print(result_df)


     operator  loading_year  loading_month  container_count  ref_id_count
0    ASI Team          2024              1              913           421
1    ASI Team          2024              2              630           318
2    ASI Team          2024              3              869           443
3    ASI Team          2024              4             1039           468
4    ASI Team          2024              5              651           304
..        ...           ...            ...              ...           ...
235   missing          2024              3                2             1
236   missing          2024              4                6             1
237   missing          2024              5                2             2
238   missing          2024              6                1             1
239   missing          2024              9                3             3

[240 rows x 5 columns]


In [12]:
# Saving the DataFrame to an Excel file
excel_filename = 'operator_volume_report.xlsx'
result_df.to_excel(excel_filename, index=False, engine='openpyxl')

print(f'DataFrame is saved to {excel_filename}')

DataFrame is saved to operator_volume_report.xlsx


In [13]:
df_merged_op

,ref_id,operator,loading_etd,to_company_id,to_company,consignee
0,C2401S089115,Benjamin Walker,2024-01-01,3A9BC7D7-1C64-4193-8AF7-699CAF87D9FD,FUYAO NORTH AMERICA INCORPORATED,FUYAO NORTH AMERICA INCORPORATED
1,C2401S088300,Kelly Anderson,2024-01-01,3D155580-F014-43F3-BA91-0E5DAC86BDD0,EASTPOINT SPORTS LTD LLC,EASTPOINT SPORTS LTD LLC
2,C2401S088637,Cara Poor,2024-01-01,F9EB87E7-7F10-4CAC-829E-A808D735A669,HERITAGE GROUP LLC,HERITAGE GROUP LLC
3,C2401S088628,Ashley Schafer,2024-01-01,5B8ABE21-CBAB-40D3-BBC5-43846A61723A,W PACKAGING LLC,W PACKAGING LLC
4,C2312S087714,Michelle Ford,2024-01-01,35177D87-D76E-447C-8FA4-BB0E97E257D6,THE UTTERMOST COMPANY,THE UTTERMOST COMPANY
...,...,...,...,...,...,...
33215,50240100055,Christina Hutchens,2024-01-04,CAA1200077,Igloo Products Corp. - Export,
33216,50240100056,Christina Hutchens,2024-01-04,CAA1200077,Igloo Products Corp. - Export,
33217,50240100057,Terry Barbour,2024-02-23,C180700042,"Westlake Chemicals & Vinyls, LLC","AXIALL, LLC"
33218,50240100058,Kim Jones,2024-07-25,C180100036,Westlake Polymers LLC,Westlake Longview Corporation


In [14]:
# Convert 'loading_etd' to datetime
df_merged_op['loading_etd'] = pd.to_datetime(df_merged_op['loading_etd'])

# Select the required columns and extract year and month from 'loading_etd'
df_result = df_merged_op[['to_company_id','to_company', 'consignee', 'operator']].copy()
df_result['year'] = df_merged_op['loading_etd'].dt.year
df_result['month'] = df_merged_op['loading_etd'].dt.month

# Display the final DataFrame
print(df_result)

                              to_company_id                        to_company  \
0      3A9BC7D7-1C64-4193-8AF7-699CAF87D9FD  FUYAO NORTH AMERICA INCORPORATED   
1      3D155580-F014-43F3-BA91-0E5DAC86BDD0          EASTPOINT SPORTS LTD LLC   
2      F9EB87E7-7F10-4CAC-829E-A808D735A669                HERITAGE GROUP LLC   
3      5B8ABE21-CBAB-40D3-BBC5-43846A61723A                   W PACKAGING LLC   
4      35177D87-D76E-447C-8FA4-BB0E97E257D6             THE UTTERMOST COMPANY   
...                                     ...                               ...   
33215                            CAA1200077     Igloo Products Corp. - Export   
33216                            CAA1200077     Igloo Products Corp. - Export   
33217                            C180700042  Westlake Chemicals & Vinyls, LLC   
33218                            C180100036             Westlake Polymers LLC   
33219                            C180700042  Westlake Chemicals & Vinyls, LLC   

                           

In [15]:
df_result

,to_company_id,to_company,consignee,operator,year,month
0,3A9BC7D7-1C64-4193-8AF7-699CAF87D9FD,FUYAO NORTH AMERICA INCORPORATED,FUYAO NORTH AMERICA INCORPORATED,Benjamin Walker,2024,1
1,3D155580-F014-43F3-BA91-0E5DAC86BDD0,EASTPOINT SPORTS LTD LLC,EASTPOINT SPORTS LTD LLC,Kelly Anderson,2024,1
2,F9EB87E7-7F10-4CAC-829E-A808D735A669,HERITAGE GROUP LLC,HERITAGE GROUP LLC,Cara Poor,2024,1
3,5B8ABE21-CBAB-40D3-BBC5-43846A61723A,W PACKAGING LLC,W PACKAGING LLC,Ashley Schafer,2024,1
4,35177D87-D76E-447C-8FA4-BB0E97E257D6,THE UTTERMOST COMPANY,THE UTTERMOST COMPANY,Michelle Ford,2024,1
...,...,...,...,...,...,...
33215,CAA1200077,Igloo Products Corp. - Export,,Christina Hutchens,2024,1
33216,CAA1200077,Igloo Products Corp. - Export,,Christina Hutchens,2024,1
33217,C180700042,"Westlake Chemicals & Vinyls, LLC","AXIALL, LLC",Terry Barbour,2024,2
33218,C180100036,Westlake Polymers LLC,Westlake Longview Corporation,Kim Jones,2024,7


In [16]:
# Saving the DataFrame to an Excel file
excel_filename = 'output_acc.xlsx'
df_result.to_excel(excel_filename, index=False, engine='openpyxl')

print(f'DataFrame is saved to {excel_filename}')

DataFrame is saved to output_acc.xlsx


In [ ]:
/Users/aashigoyal/Desktop/vscode/aashimain/agl/db_col.xlsm